In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Vertex AI 构建强化学习应用程序的逐步指南

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/community-content/tf_agents_bandits_movie_recommendation_with_kfp_and_vertex_sdk/step_by_step_sdk_tf_agents_bandits_movie_recommendation/step_by_step_sdk_tf_agents_bandits_movie_recommendation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab上运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/community-content/tf_agents_bandits_movie_recommendation_with_kfp_and_vertex_sdk/step_by_step_sdk_tf_agents_bandits_movie_recommendation/step_by_step_sdk_tf_agents_bandits_movie_recommendation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>

## 概述
该演示展示了在构建电影推荐系统时使用[TF-Agents](https://www.tensorflow.org/agents)和[Vertex AI](https://cloud.google.com/vertex-ai)的情况，利用强化学习。该演示适用于希望使用TensorFlow和TF-Agents库创建强化学习应用程序的开发人员，利用Vertex AI服务（包括自定义训练、自定义预测、在托管端点上部署模型和提取预测）。建议开发人员对基本强化学习理论有所了解，特别是上下文匪徒的公式以及TF-Agents接口。请注意，上下文匪徒形成了RL的一种特殊情况，其中代理器采取的行动不会改变环境的状态。“上下文”指的是代理在了解上下文（环境观察）的情况下从一组动作中选择。

### 数据集
该演示使用[MovieLens 100K](https://www.kaggle.com/prajitdatta/movielens-100k-dataset)数据集来模拟具有用户及其偏好的环境。它位于`gs://cloud-samples-data/vertex-ai/community-content/tf_agents_bandits_movie_recommendation_with_kfp_and_vertex_sdk/u.data`。

### 目标
在本笔记本中，您将学习如何使用Vertex AI的自定义训练、自定义预测和端点部署服务构建基于TF-Agents（特别是匪类模块）的强化应用程序。
对于自定义训练，您将实施基于策略的训练，在该训练中您将与基于MovieLens的模拟环境进行交互，以（1）获取环境观察、（2）根据给定观察选择动作使用数据收集策略，以及（3）获取与（1）（2）对应的奖励形式的环境反馈。这些数据片段形成训练数据记录。此过程与离线训练不同，离线训练中您不一定与策略实际输出的操作相关联的训练数据。

此演示由2个主要步骤组成：
1. 在本地运行，使用[TF-Agents](https://www.tensorflow.org/agents)实现。
2. 在[Vertex AI](https://cloud.google.com/vertex-ai)上执行。

除了培训、预测和预测工作流程外，该演示还展示以下优化技术：
1. 使用Vertex AI进行超参数调整
2. 使用TensorBoard Profiler对训练过程和资源进行配置分析，可为加速改进、扩展等目的提供信息

该演示参考了来自以下代码的内容：[此TF-Agents示例](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/examples/v2/train_eval_movielens.py)、[此Vertex AI SDK自定义容器训练示例](https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/unofficial/sdk/AI_Platform_(Unified)_SDK_BigQuery_Custom_Container_Training.ipynb)和[此Vertex AI SDK自定义容器预测示例](https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/unofficial/sdk/AI_Platform_(Unified)_SDK_Custom_Container_Prediction.ipynb)。

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Build
* Cloud Storage
* Container Registry

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)、[Cloud Build价格](https://cloud.google.com/build/pricing)、[Cloud Storage价格](https://cloud.google.com/storage/pricing)和[Container Registry价格](https://cloud.google.com/container-registry/pricing)，使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

### 设置本地开发环境

**如果您正在使用Colab或Google Cloud笔记本**，您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简化的说明：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端shell中命令行运行`pip3 install jupyter`。

5. 要启动Jupyter，请在终端shell中命令行运行`jupyter notebook`。

6. 在Jupyter Notebook Dashboard中打开此笔记本。

安装额外的包

安装在您的笔记本环境中尚未安装的额外包依赖，例如 AI 平台 SDK 和 TF-Agents。请使用每个包的最新主要 GA 版本。

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
! pip3 install {USER_FLAG} google-cloud-aiplatform
! pip3 install {USER_FLAG} google-cloud-storage
! pip3 install {USER_FLAG} numpy
! pip3 install {USER_FLAG} cloudml-hypertune
! pip3 install {USER_FLAG} --upgrade tensorflow
! pip3 install {USER_FLAG} --upgrade pillow
! pip3 install {USER_FLAG} --upgrade tf-agents
! pip3 install {USER_FLAG} --upgrade tensorboard-plugin-profile

重新启动内核

安装额外的软件包后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在开始之前

### 选择GPU运行时

**确保如果有这个选项的话，您正在一个GPU运行时中运行这个笔记本。在Colab中，选择“运行时 --> 更改运行时类型 > GPU”**

### 设置您的谷歌云项目

**不管您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用，用于支付计算/存储成本。

2. [确保为您的项目启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API、Cloud Build API、Cloud Storage API 和 Container Registry API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,cloudbuild.googleapis.com,storage.googleapis.com,containerregistry.googleapis.com)。

4. 如果您在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保
Cloud SDK为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 运行以 `!` 开头的行作为shell命令，并将以 `$` 开头的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
import os

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，请在这里设置您的项目ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

时间戳

如果您正在参加现场教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的Google云账户

**如果您正在使用Google云笔记本**，您的环境已经经过验证。请跳过此步骤。

如果您正在使用Colab，请运行下面的单元格，并按提示进行身份验证，通过oAuth认证您的帐户。

否则，请按照以下步骤操作：

1. 在Cloud控制台中，转到[**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

4. 在**授予此服务账户对项目的访问权限**部分，点击**角色**下拉列表。在筛选框中输入"Vertex AI"，并选择**Vertex AI管理员**。在筛选框中输入"Storage Object Admin"，并选择**Storage Object Admin**。

5. 点击*创建*。一个包含您密钥的JSON文件将下载到您的本地环境中。

6. 在下面的单元格中，将您的服务帐号密钥的路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量输入，并运行该单元格。

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth
        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储存储桶

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

在本教程中，一个云存储存储桶保存MovieLens数据集文件，用于模型训练。Vertex AI还会将训练作业生成的经过训练的模型保存在同一个存储桶中。使用这个模型工件，您可以创建Vertex AI模型和端点资源，以便提供在线预测。

在下面设置您的云存储存储桶的名称。它必须在所有云存储存储桶中是唯一的。

您还可以更改`REGION`变量，该变量在本笔记本的其余部分中使用。确保在[选择可用Vertex AI服务的地区](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions)时选择一个地区。您不能使用多区域存储存储桶进行与Vertex AI的训练。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"} The bucket should be in same region as uCAIP. The bucket should not be multi-regional for custom training jobs to work.
REGION = "[your-region]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问权限：

In [ ]:
! gsutil ls -al $BUCKET_NAME

导入库并定义常量

In [ ]:
import functools
import json
import os
from collections import defaultdict
from typing import Callable, Dict, List, Optional, TypeVar

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from google.cloud import aiplatform, storage
from tf_agents.agents import TFAgent
from tf_agents.bandits.agents import lin_ucb_agent
from tf_agents.bandits.agents.examples.v2 import trainer
from tf_agents.bandits.environments import (environment_utilities,
                                            movielens_py_environment)
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import TFEnvironment, tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.metrics.tf_metric import TFStepMetric
from tf_agents.policies import policy_saver

if tf.__version__[0] != "2":
    raise Exception("The trainer only runs with TensorFlow version 2.")

T = TypeVar("T")

In [ ]:
ROOT_DIR = f"{BUCKET_NAME}/artifacts"  # @param {type:"string"} Root directory for writing logs/summaries/checkpoints.
ARTIFACTS_DIR = f"{BUCKET_NAME}/artifacts"  # @param {type:"string"} Where the trained model will be saved and restored.
PROFILER_DIR = f"{BUCKET_NAME}/profiler"  # @param {type:"string"} Directory for TensorBoard Profiler artifacts.
DATA_PATH = f"{BUCKET_NAME}/artifacts/u.data"  # Location of the MovieLens 100K dataset's "u.data" file.
RAW_BUCKET_NAME = BUCKET_NAME[5:]  # Remove the prefix `gs://`.

In [ ]:
# Copy the sample data into your DATA_PATH
! gsutil cp "gs://cloud-samples-data/vertex-ai/community-content/tf_agents_bandits_movie_recommendation_with_kfp_and_vertex_sdk/u.data"  $DATA_PATH

In [ ]:
# Set hyperparameters.
BATCH_SIZE = 8  # @param {type:"integer"} Training and prediction batch size.
TRAINING_LOOPS = 5  # @param {type:"integer"} Number of training iterations.
STEPS_PER_LOOP = 2  # @param {type:"integer"} Number of driver steps per training iteration.

# Set MovieLens simulation environment parameters.
RANK_K = 20  # @param {type:"integer"} Rank for matrix factorization in the MovieLens environment; also the observation dimension.
NUM_ACTIONS = 20  # @param {type:"integer"} Number of actions (movie items) to choose from.
PER_ARM = False  # Use the non-per-arm version of the MovieLens environment.

# Set agent parameters.
TIKHONOV_WEIGHT = 0.001  # @param {type:"number"} LinUCB Tikhonov regularization weight.
AGENT_ALPHA = 10.0  # @param {type:"number"} LinUCB exploration parameter that multiplies the confidence intervals.

实施和本地执行（可选）

定义RL模块[局部]

定义一个[MovieLens特定的赌博环境]，一个[线性UCB代理]和[遗憾度量]。

In [ ]:
# Define RL environment.
env = movielens_py_environment.MovieLensPyEnvironment(
    DATA_PATH, RANK_K, BATCH_SIZE, num_movies=NUM_ACTIONS, csv_delimiter="\t")
environment = tf_py_environment.TFPyEnvironment(env)

# Define RL agent/algorithm.
agent = lin_ucb_agent.LinearUCBAgent(
    time_step_spec=environment.time_step_spec(),
    action_spec=environment.action_spec(),
    tikhonov_weight=TIKHONOV_WEIGHT,
    alpha=AGENT_ALPHA,
    dtype=tf.float32,
    accepts_per_arm_features=PER_ARM)
print("TimeStep Spec (for each batch):\n", agent.time_step_spec, "\n")
print("Action Spec (for each batch):\n", agent.action_spec, "\n")
print("Reward Spec (for each batch):\n", environment.reward_spec(), "\n")

# Define RL metric.
optimal_reward_fn = functools.partial(
    environment_utilities.compute_optimal_reward_with_movielens_environment,
    environment=environment)
regret_metric = tf_bandit_metrics.RegretMetric(optimal_reward_fn)
metrics = [regret_metric]

### 在本地训练模型

定义训练逻辑（on-policy训练）。以下函数与[trainer.train](https://github.com/tensorflow/agents/blob/r0.8.0/tf_agents/bandits/agents/examples/v2/trainer.py#L104)相同，但它会跟踪中间指标值并将不同的工件保存到不同的位置。您也可以直接调用[trainer.train](https://github.com/tensorflow/agents/blob/r0.8.0/tf_agents/bandits/agents/examples/v2/trainer.py#L104)，它也会训练策略。

In [ ]:
def train(
    root_dir: str,
    agent: TFAgent,
    environment: TFEnvironment,
    training_loops: int,
    steps_per_loop: int,
    additional_metrics: Optional[List[TFStepMetric]] = None,
    training_data_spec_transformation_fn: Optional[Callable[[T], T]] = None,
) -> Dict[str, List[float]]:
    """Performs `training_loops` iterations of training on the agent's policy.

    Uses the `environment` as the problem formulation and source of immediate
    feedback and the agent's algorithm, to perform `training-loops` iterations
    of on-policy training on the policy.
    If one or more baseline_reward_fns are provided, the regret is computed
    against each one of them. Here is example baseline_reward_fn:
    def baseline_reward_fn(observation, per_action_reward_fns):
        rewards = ... # compute reward for each arm
        optimal_action_reward = ... # take the maximum reward
        return optimal_action_reward

    Args:
        root_dir: Path to the directory where training artifacts are written.
        agent: An instance of `TFAgent`.
        environment: An instance of `TFEnvironment`.
        training_loops: An integer indicating how many training loops should be run.
        steps_per_loop: An integer indicating how many driver steps should be
           executed and presented to the trainer during each training loop.
        additional_metrics: Optional; list of metric objects to log, in addition to
          default metrics `NumberOfEpisodes`, `AverageReturnMetric`, and
          `AverageEpisodeLengthMetric`.
        training_data_spec_transformation_fn: Optional; function that transforms
          the data items before they get to the replay buffer.

    Returns:
        A dict mapping metric names (eg. "AverageReturnMetric") to a list of
        intermediate metric values over `training_loops` iterations of training.
    """
    if training_data_spec_transformation_fn is None:
        data_spec = agent.policy.trajectory_spec
    else:
        data_spec = training_data_spec_transformation_fn(
            agent.policy.trajectory_spec)
    replay_buffer = trainer.get_replay_buffer(data_spec, environment.batch_size,
                                              steps_per_loop)

    # `step_metric` records the number of individual rounds of bandit interaction;
    # that is, (number of trajectories) * batch_size.
    step_metric = tf_metrics.EnvironmentSteps()
    metrics = [
        tf_metrics.NumberOfEpisodes(),
        tf_metrics.AverageEpisodeLengthMetric(batch_size=environment.batch_size)
    ]
    if additional_metrics:
        metrics += additional_metrics

    if isinstance(environment.reward_spec(), dict):
        metrics += [tf_metrics.AverageReturnMultiMetric(
            reward_spec=environment.reward_spec(),
            batch_size=environment.batch_size)]
    else:
        metrics += [
            tf_metrics.AverageReturnMetric(batch_size=environment.batch_size)]

    # Store intermediate metric results, indexed by metric names.
    metric_results = defaultdict(list)

    if training_data_spec_transformation_fn is not None:
        def add_batch_fn(data): return replay_buffer.add_batch(training_data_spec_transformation_fn(data)) 
        
    else:
        add_batch_fn = replay_buffer.add_batch

    observers = [add_batch_fn, step_metric] + metrics

    driver = dynamic_step_driver.DynamicStepDriver(
        env=environment,
        policy=agent.collect_policy,
        num_steps=steps_per_loop * environment.batch_size,
        observers=observers)

    training_loop = trainer.get_training_loop_fn(
        driver, replay_buffer, agent, steps_per_loop)
    saver = policy_saver.PolicySaver(agent.policy)

    for _ in range(training_loops):
        training_loop()
        metric_utils.log_metrics(metrics)
        for metric in metrics:
            metric.tf_summaries(train_step=step_metric.result())
            metric_results[type(metric).__name__].append(metric.result().numpy())
    saver.save(root_dir)
    return metric_results

训练RL策略并收集中间指标结果。同时，使用[TensorBoard Profiler](https://www.tensorflow.org/guide/profiler)来对训练过程和资源进行剖析。

In [ ]:
tf.profiler.experimental.start(PROFILER_DIR)

metric_results = train(
    root_dir=ROOT_DIR,
    agent=agent,
    environment=environment,
    training_loops=TRAINING_LOOPS,
    steps_per_loop=STEPS_PER_LOOP,
    additional_metrics=metrics)

tf.profiler.experimental.stop()

### 评估RL指标[本地]

您可以可视化遗憾和平均回报指标随着训练步骤的演变。

In [ ]:
def plot(metric_results, metric_name):
    plt.plot(metric_results[metric_name])
    plt.ylabel(metric_name)
    plt.xlabel("Step")
    plt.title("{} versus Step".format(metric_name))

In [ ]:
plot(metric_results, "RegretMetric")

In [ ]:
plot(metric_results, "AverageReturnMetric")

加载[TensorBoard Profiler](https://www.tensorflow.org/guide/profiler)相关内容，以进行培训过程和资源的数据分析。可视化不同设备上的操作统计信息、操作追踪等信息。这些信息可以帮助您识别培训性能中的瓶颈，并指导您对速度和/或可扩展性的潜在改进。

In [ ]:
# If on Google Cloud Notebooks, then don't execute this code.
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:

        # Load the TensorBoard notebook extension.
        %load_ext tensorboard

In [ ]:
# If on Google Cloud Notebooks, then don't execute this code.
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:

        %tensorboard --logdir $PROFILER_DIR

对于Google Cloud笔记本，您可以执行以下操作：

1. 从GCP控制台打开[Cloud Shell](https://cloud.google.com/shell)。
2. 安装依赖项：`pip3 install tensorflow==2.5.0 tensorboard-plugin-profile==2.5.0`。
3. 运行以下命令：`tensorboard --logdir <PROFILER_DIR>`。您将看到输出消息 "TensorBoard 2.5.0 at http://localhost:<PORT>/（按下CTRL+C退出）"。记下端口号。
4. 您可以单击[Web Preview](https://cloud.google.com/shell/docs/using-web-preview)按钮并查看TensorBoard仪表板和性能分析结果。您需要将Web Preview的端口配置为与第3步中收到的端口相同。

在Vertex AI中执行

该部分包括以下步骤：
1. 对`policy_util`和`task`模块运行单元测试
2. 创建超参数调整和训练自定义容器
3. 提交超参数调整作业 [可选]
4. 创建自定义预测容器
5. 提交自定义容器训练作业
6. 部署训练好的模型到终端
7. 在终端上进行预测

在`policy_util`和`task`模块上运行单元测试

在`src/training/`中运行模块的单元测试。

在`src/tests/`中找到测试，并在测试文件中填写标有“FILL IN”的配置。

In [ ]:
! python3 -m unittest src/tests/test_policy_util.py

In [ ]:
! python3 -m unittest src/tests/test_task.py

### 创建超参数调整和训练自定义容器

创建一个自定义容器，可用于超参数调整和训练。相关的源代码位于`src/training/`目录下。这将作为自定义容器的内部脚本。

与之前一样，训练函数与[trainer.train](https://github.com/tensorflow/agents/blob/r0.8.0/tf_agents/bandits/agents/examples/v2/trainer.py#L104)相同，但它会跟踪中间度量值，支持超参数调整，并（针对训练）将工件保存到不同的位置。超参数调整和训练的训练逻辑相同。

#### 执行超参数调整：
- 代码不保存模型工件。它从Vertex AI超参数调整服务接收命令行参数作为超参数值，并使用cloudml-hypertune在每次试验时向Vertex AI报告训练结果度量值。
- 请注意，如果决定保存模型工件，将它们保存在相同目录可能会在超参数调整作业中使用并行试验时导致覆盖错误。推荐的方法是将每次试验的工件保存到不同的子目录。这也可以帮助您从不同试验中恢复所有工件，并潜在地避免重新训练。
- 在[这里](https://cloud.google.com/vertex-ai/docs/training/containers-overview#hyperparameter_tuning_with_custom_containers)阅读有关自定义容器的超参数调整更多信息；在[这里](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)阅读有关超参数调整支持的更多信息。

#### 执行训练：
- 该代码将模型工件保存到`os.environ["AIP_MODEL_DIR"]`以及`ARTIFACTS_DIR`，如此[所需](https://github.com/googleapis/python-aiplatform/blob/v0.8.0/google/cloud/aiplatform/training_jobs.py#L2202)。
- 如果您想对函数进行更改，请确保仍将训练的策略保存为SavedModel以清理目录，并避免保存检查点和其他工件，以便将模型部署到终端处理工作。

In [ ]:
HPTUNING_TRAINING_CONTAINER = "hptuning-training-custom-container"  # @param {type:"string"} Name of the container image.

创建一个 Cloud Build YAML 文件

使用[Kaniko](https://github.com/GoogleContainerTools/kaniko)构建超参数调整/训练容器。您可以应用缓存并指定构建机器类型。另外，您也可以使用 Docker 构建。

In [ ]:
cloudbuild_yaml = """steps:
- name: 'gcr.io/kaniko-project/executor:latest'
  args: ['--destination=gcr.io/{PROJECT_ID}/{HPTUNING_TRAINING_CONTAINER}:latest',
         '--cache=true',
         '--cache-ttl=99h']
options:
  machineType: 'E2_HIGHCPU_8'""".format(
    PROJECT_ID=PROJECT_ID,
    HPTUNING_TRAINING_CONTAINER=HPTUNING_TRAINING_CONTAINER,
)

with open("cloudbuild.yaml", "w") as fp:
    fp.write(cloudbuild_yaml)

#### 编写一个Dockerfile

- 使用[cloudml-hypertune](https://github.com/GoogleCloudPlatform/cloudml-hypertune) Python包来将训练指标报告给Vertex AI以进行超参数调优。
- 使用Google [Cloud Storage客户端库](https://cloud.google.com/storage/docs/reference/libraries)在训练期间读取从先前的超参数调优作业中学到的最佳超参数。

In [ ]:
%%writefile Dockerfile

# Specifies base image and tag.
FROM gcr.io/google-appengine/python
WORKDIR /root

# Installs additional packages.
RUN pip3 install cloudml-hypertune==0.1.0.dev6
RUN pip3 install google-cloud-storage==1.39.0
RUN pip3 install tensorflow==2.5.0
RUN pip3 install tensorboard-plugin-profile==2.5.0
RUN pip3 install tf-agents==0.8.0
RUN pip3 install matplotlib==3.4.2

# Copies training code to the Docker image.
COPY src/training /root/src/training

# Sets up the entry point to invoke the task.
ENTRYPOINT ["python3", "-m", "src.training.task"]

使用云构建构建定制容器

In [ ]:
! gcloud builds submit --config cloudbuild.yaml

### 提交超参数调整作业 [可选]

- 使用自定义容器提交一个超参数训练作业。阅读更多关于在示例中使用 Python 包作为替代使用自定义容器的详细信息 [这里](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning#create)。
- 定义超参数、最大试验数量、并行试验数量、参数搜索算法、机器规格、加速器、工作池等。

In [ ]:
RUN_HYPERPARAMETER_TUNING = True  # Execute hyperparameter tuning instead of regular training.
TRAIN_WITH_BEST_HYPERPARAMETERS = False  # Do not train.

HPTUNING_RESULT_DIR = "hptuning/"  # @param {type: "string"} Directory to store the best hyperparameter(s) in `BUCKET_NAME` and locally (temporarily).
HPTUNING_RESULT_PATH = os.path.join(HPTUNING_RESULT_DIR, "result.json")  # @param {type: "string"} Path to the file containing the best hyperparameter(s).

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

In [ ]:
def create_hyperparameter_tuning_job_sample(
    project: str,
    display_name: str,
    image_uri: str,
    args: List[str],
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com"
) -> None:
    """Creates a hyperparameter tuning job using a custom container.

    Args:
        project: GCP project ID.
        display_name: GCP console display name for the hyperparameter tuning job in
            Vertex AI.
        image_uri: URI to the hyperparameter tuning container image in Container
            Registry.
        args: Arguments passed to the container.
        location: Service location.
        api_endpoint: API endpoint, eg. `<location>-aiplatform.googleapis.com`.

    Returns:
        A string of the hyperparameter tuning job ID.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.JobServiceClient(client_options=client_options)

    # study_spec
    # Metric based on which to evaluate which combination of hyperparameter(s) to choose
    metric = {
        "metric_id": "final_average_return",  # Metric you report to Vertex AI.
        "goal": aiplatform.gapic.StudySpec.MetricSpec.GoalType.MAXIMIZE,
    }

    # Hyperparameter(s) to tune
    training_loops = {
        "parameter_id": "training-loops",
        "discrete_value_spec": {"values": [4, 16]},
        "scale_type": aiplatform.gapic.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
    }
    steps_per_loop = {
        "parameter_id": "steps-per-loop",
        "discrete_value_spec": {"values": [1, 2]},
        "scale_type": aiplatform.gapic.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
    }

    # trial_job_spec
    machine_spec = {
        "machine_type": "n1-standard-4",
        "accelerator_type": aiplatform.gapic.AcceleratorType.ACCELERATOR_TYPE_UNSPECIFIED,
        "accelerator_count": None,
    }
    worker_pool_spec = {
        "machine_spec": machine_spec,
        "replica_count": 1,
        "container_spec": {
            "image_uri": image_uri,
            "args": args,
        },
    }

    # hyperparameter_tuning_job
    hyperparameter_tuning_job = {
        "display_name": display_name,
        "max_trial_count": 4,
        "parallel_trial_count": 2,
        "study_spec": {
            "metrics": [metric],
            "parameters": [training_loops, steps_per_loop],
            "algorithm": aiplatform.gapic.StudySpec.Algorithm.RANDOM_SEARCH,
        },
        "trial_job_spec": {"worker_pool_specs": [worker_pool_spec]},
    }
    parent = f"projects/{project}/locations/{location}"

    # Create job
    response = client.create_hyperparameter_tuning_job(
        parent=parent,
        hyperparameter_tuning_job=hyperparameter_tuning_job)
    job_id = response.name.split("/")[-1]
    print("Job ID:", job_id)
    print("Job config:", response)

    return job_id

In [ ]:
args = [
    f"--data-path={DATA_PATH}",
    f"--batch-size={BATCH_SIZE}",
    f"--rank-k={RANK_K}",
    f"--num-actions={NUM_ACTIONS}",
    f"--tikhonov-weight={TIKHONOV_WEIGHT}",
    f"--agent-alpha={AGENT_ALPHA}",
]
if RUN_HYPERPARAMETER_TUNING:
    args.append("--run-hyperparameter-tuning")
elif TRAIN_WITH_BEST_HYPERPARAMETERS:
    args.append("--train-with-best-hyperparameters")

In [ ]:
job_id = create_hyperparameter_tuning_job_sample(
    project=PROJECT_ID,
    display_name="movielens-hyperparameter-tuning-job",
    image_uri=f"gcr.io/{PROJECT_ID}/{HPTUNING_TRAINING_CONTAINER}:latest",
    args=args,
    location=REGION,
    api_endpoint=f"{REGION}-aiplatform.googleapis.com")

完成此操作大约需要 ~20 分钟。

#### 检查超参数调整作业状态

- 详细了解如何管理作业 [点击这里](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning#manage)。

In [ ]:
def get_hyperparameter_tuning_job_sample(
    project: str,
    hyperparameter_tuning_job_id: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
) -> aiplatform.HyperparameterTuningJob:
    """Gets the current status of a hyperparameter tuning job.

    Args:
        project: GCP project ID.
        hyperparameter_tuning_job_id: Hyperparameter tuning job ID.
        location: Service location.
        api_endpoint: API endpoint, eg. `<location>-aiplatform.googleapis.com`.

    Returns:
        Details of the hyperparameter tuning job, such as its running status,
        results of its trials, etc.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.JobServiceClient(client_options=client_options)
    name = client.hyperparameter_tuning_job_path(
        project=project,
        location=location,
        hyperparameter_tuning_job=hyperparameter_tuning_job_id)
    response = client.get_hyperparameter_tuning_job(name=name)
    return response

In [ ]:
trials = None
while True:
    response = get_hyperparameter_tuning_job_sample(
        project=PROJECT_ID,
        hyperparameter_tuning_job_id=job_id,
        location=REGION,
        api_endpoint=f"{REGION}-aiplatform.googleapis.com")
    if response.state.name == 'JOB_STATE_SUCCEEDED':
        print("Job succeeded.\nJob Time:", response.update_time - response.create_time)
        trials = response.trials
        print("Trials:", trials)
        break
    elif response.state.name == "JOB_STATE_FAILED":
        print("Job failed.")
        break
    elif response.state.name == "JOB_STATE_CANCELLED":
        print("Job cancelled.")
    break
    else:
        print(f"Current job status: {response.state.name}.")
    time.sleep(60)

寻找每个度量标准的最佳参数组合

In [ ]:
if trials:
    # Dict mapping from metric names to the best metric values seen so far
    best_objective_values = dict.fromkeys(
        [metric.metric_id for metric in trials[0].final_measurement.metrics],
        -np.inf)
    # Dict mapping from metric names to a list of the best combination(s) of
    # hyperparameter(s). Each combination is a dict mapping from hyperparameter
    # names to their values.
    best_params = defaultdict(list)
    for trial in trials:
        # `final_measurement` and `parameters` are `RepeatedComposite` objects.
        # Reference the structure above to extract the value of your interest.
        for metric in trial.final_measurement.metrics:
            params = {
                param.parameter_id: param.value for param in trial.parameters}
            if metric.value > best_objective_values[metric.metric_id]:
                best_params[metric.metric_id] = [params]
            elif metric.value == best_objective_values[metric.metric_id]:
                best_params[param.parameter_id].append(params)  # Handle cases where multiple hyperparameter values lead to the same performance.
    print("Best hyperparameter value(s):")
    for metric, params in best_params.items():
        print(f"Metric={metric}: {sorted(params)}")
else:
    print("No hyperparameter tuning job trials found.")

将感兴趣的度量标准的最佳超参数组合转换为JSON

In [ ]:
! mkdir $HPTUNING_RESULT_DIR

with open(HPTUNING_RESULT_PATH, "w") as f:
    json.dump(best_params["final_average_return"][0], f)

上传最佳超参数到GCS以用于训练

In [ ]:
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(RAW_BUCKET_NAME)
blob = bucket.blob(HPTUNING_RESULT_PATH)
blob.upload_from_filename(HPTUNING_RESULT_PATH)

### 创建自定义预测容器

与训练一样，创建一个自定义预测容器。此容器处理与常规 TensorFlow 模型不同的 TF-Agents 特定逻辑。具体来说，它使用训练过的策略找到预测的动作。相关的源代码位于 `src/prediction/`。
查看 Vertex AI 预测的其他选项[此处](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions)。

#### 提供预测：
- 使用 [`tensorflow.saved_model.load`](https://www.tensorflow.org/agents/api_docs/python/tf_agents/policies/PolicySaver#usage) 而不是 [`tf_agents.policies.policy_loader.load`](https://github.com/tensorflow/agents/blob/r0.8.0/tf_agents/policies/policy_loader.py#L26) 来加载训练过的策略，因为后者产生的对象类型是 [`SavedModelPyTFEagerPolicy`](https://github.com/tensorflow/agents/blob/402b8aa81ca1b578ec1f687725d4ccb4115386d2/tf_agents/policies/py_tf_eager_policy.py#L137)，其 `action()` 方法不适用于此处。
- 请注意，预测请求仅包含观察数据而不包含奖励。这是因为：预测任务是一个独立的请求，不需要对系统状态有先前的了解。与此同时，最终用户只知道他们在此刻观察到的内容。奖励是在动作执行后产生的信息，因此最终用户不会知道奖励。在处理预测请求时，您需要创建一个 [`TimeStep`](https://www.tensorflow.org/agents/api_docs/python/tf_agents/trajectories/TimeStep) 对象（包含 `observation`、`reward`、`discount`、`step_type`），使用 [`restart()`](https://www.tensorflow.org/agents/api_docs/python/tf_agents/trajectories/restart) 函数传入一个 `observation`。此函数会创建轨迹步骤中的第一个 TimeStep，在这个步骤中，奖励为0，折扣为1，步骤类型标记为第一个时间步。换句话说，每个预测请求形成一个新轨迹中的第一个 `TimeStep`。
- 对于预测响应，请避免使用 NumPy 类型的值；相反，使用诸如 [`tolist()`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.tolist.html) 这样的方法将其转换为本机 Python 值，而不是 `list()`。
- `src/prediction` 目录中存在一个预启动脚本。FastAPI 在启动服务器之前执行此脚本。`PORT` 环境变量被设置为等于 `AIP_HTTP_PORT`，以便在 Vertex AI 期望的相同端口上运行 FastAPI。

In [ ]:
PREDICTION_CONTAINER = "prediction-custom-container"  # @param {type:"string"} Name of the container image.

创建一个Cloud Build YAML文件

使用[Kaniko](https://github.com/GoogleContainerTools/kaniko)来构建自定义预测容器。

In [ ]:
cloudbuild_yaml = """steps:
- name: 'gcr.io/kaniko-project/executor:latest'
  args: ['--destination=gcr.io/{PROJECT_ID}/{PREDICTION_CONTAINER}:latest',
         '--cache=true',
         '--cache-ttl=99h']
  env: ['AIP_STORAGE_URI={ARTIFACTS_DIR}']
options:
  machineType: 'E2_HIGHCPU_8'""".format(
    PROJECT_ID=PROJECT_ID,
    PREDICTION_CONTAINER=PREDICTION_CONTAINER,
    ARTIFACTS_DIR=ARTIFACTS_DIR
)

with open("cloudbuild.yaml", "w") as fp:
    fp.write(cloudbuild_yaml)

定义依赖关系

- 请注意，这些依赖关系应该彼此兼容（例如，tensorflow==2.5.0 需要 numpy<=1.19.2）。

In [ ]:
%%writefile requirements.txt

numpy~=1.19.2
six~=1.15.0
typing-extensions~=3.7.4
pillow==9.0.0
tf-agents==0.8.0
tensorflow==2.5.0

写一个Dockerfile

注意：保留服务器目录`app`。

In [ ]:
%%writefile Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7

COPY src/prediction /app
COPY requirements.txt /app/requirements.txt

RUN pip3 install -r /app/requirements.txt

使用Cloud Build构建预测容器

In [ ]:
! gcloud builds submit --config cloudbuild.yaml

### 提交自定义容器训练作业

- 请再次注意，存储桶必须与服务位置位于同一地区，并且不应该是多区域的。
- 更多关于CustomContainerTrainingJob的源代码，请访问[此处](https://github.com/googleapis/python-aiplatform/blob/v0.8.0/google/cloud/aiplatform/training_jobs.py#L2153)。
- 与本地执行类似，您可以使用TensorBoard Profiler来跟踪训练进程和资源，并使用以下命令可视化相应的工件： `%tensorboard --logdir $PROFILER_DIR`。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

In [ ]:
RUN_HYPERPARAMETER_TUNING = False  # Execute regular training instead of hyperparameter tuning.
TRAIN_WITH_BEST_HYPERPARAMETERS = True  # @param {type:"bool"} Whether to use learned hyperparameters in training.

In [ ]:
args = [
    f"--artifacts-dir={ARTIFACTS_DIR}",
    f"--profiler-dir={PROFILER_DIR}",
    f"--data-path={DATA_PATH}",
    f"--batch-size={BATCH_SIZE}",
    f"--rank-k={RANK_K}",
    f"--num-actions={NUM_ACTIONS}",
    f"--tikhonov-weight={TIKHONOV_WEIGHT}",
    f"--agent-alpha={AGENT_ALPHA}",
]
if RUN_HYPERPARAMETER_TUNING:
    args.append("--run-hyperparameter-tuning")
elif TRAIN_WITH_BEST_HYPERPARAMETERS:
    args.append("--train-with-best-hyperparameters")
    args.append(f"--best-hyperparameters-bucket={RAW_BUCKET_NAME}")
    args.append(f"--best-hyperparameters-path={HPTUNING_RESULT_PATH}")

In [ ]:
job = aiplatform.CustomContainerTrainingJob(
    display_name="train-movielens",
    container_uri=f"gcr.io/{PROJECT_ID}/{HPTUNING_TRAINING_CONTAINER}:latest",
    command=["python3", "-m", "src.training.task"] + args,  # Pass in training arguments, including hyperparameters.
    model_serving_container_image_uri=f"gcr.io/{PROJECT_ID}/{PREDICTION_CONTAINER}:latest",
    model_serving_container_predict_route="/predict",
    model_serving_container_health_route="/health")

print("Training Spec:", job._managed_model)

model = job.run(
    model_display_name="movielens-model",
    replica_count=1,
    machine_type="n1-standard-4",
    accelerator_type="ACCELERATOR_TYPE_UNSPECIFIED",
    accelerator_count=0)

In [ ]:
print("Model display name:", model.display_name)
print("Model ID:", model.name)

部署训练模型到一个端点

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

In [ ]:
print("Endpoint display name:", endpoint.display_name)
print("Endpoint ID:", endpoint.name)

### 在终端上进行预测
- 把预测输入放入一个名为`instances`的列表中。观察结果应该是维度为(BATCH_SIZE, RANK_K)的。在这里阅读更多关于MovieLens仿真环境观察的信息：(https://github.com/tensorflow/agents/blob/v0.8.0/tf_agents/bandits/environments/movielens_py_environment.py#L32-L138)。
- 在这里阅读更多关于终端预测API的信息：(https://cloud.google.com/sdk/gcloud/reference/ai/endpoints/predict)。

In [ ]:
endpoint.predict(
    instances=[
        {"observation": [list(np.ones(20)) for _ in range(8)]},
    ]
)

## 概要

### 电影评价（MovieLens）模拟环境的目的是什么？

电影评价（MovieLens）环境*模拟*了包含用户及其各自偏好的真实世界环境。在内部，电影评价（MovieLens）模拟环境接收用户对电影项目的评级矩阵，并对该评级矩阵执行`RANK_K`矩阵分解，以解决矩阵的稀疏性问题。完成构建步骤后，环境可以生成维度为`RANK_K`的用户向量来代表模拟环境中的用户，并能够确定任何用户和电影项目对之间的近似奖励。在强化学习的术语中，用户向量称为观察值，推荐的电影项目称为行动，近似评级则称为奖励。因此，这个环境定义了面临的强化学习问题：如何推荐电影以最大化用户评级，在一个模拟世界中的用户，其偏好由电影评价（MovieLens）数据集定义，同时没有关于环境内部机制的任何知识。

需要注意的是，用户向量可能不与原始评级矩阵中的维度相同，并且近似评级（以解决评级数据的稀疏性）可能不等同于原始评级。用户向量中的各个条目并不对应于真实世界的含义，比如用户年龄等。在预测请求中，观察值是与电影评价（MovieLens）模拟环境生成的用户向量相同空间中的用户向量。换句话说，它们代表用户的方式与电影评价（MovieLens）环境生成的用户向量/观察值相同。

这个演示采用电影评价（MovieLens）环境是因为可以基于公共数据集而无需与真实世界通信；这种通信会给演示必需的步骤增加额外负担，很可能依赖于难以推广到您产品需求的特定实现。

### 如何将这个演示应用于生产环境

#### 步骤 0：演示

通过使用电影评价（MovieLens）模拟环境来了解这个演示。

#### 步骤 1：离线模拟

为了评估您的强化学习模型的性能，您可能需要首先运行离线模拟，以确定您的强化学习模型是否符合生产要求。在这种情况下，您可以拥有一个静态数据集，类似于电影评价（MovieLens）数据集但可能更大，并且您可以构建一个自定义模拟环境来代替电影评价（MovieLens）模拟环境。在自定义环境中，您可以决定如何构建观察值和奖励，例如如何使用用户向量来表示用户及这些向量的样子，也许通过神经网络中的嵌入层。您可以像对待电影评价（MovieLens）一样应用剩余步骤和代码，然后评估您的模型。完成离线模拟后，您可以继续进行启动模型的下一步，例如进行A/B测试。

#### 步骤 2：真实世界系统

当您在生产环境中部署这个演示的步骤时，您将需要用真实世界系统或与真实世界对接的通信机制替换电影评价（MovieLens）模拟环境。在训练中，您从真实世界环境中提取用户向量/观察值和评分/奖励。此时，用户向量中的各个条目可能具有实际含义，比如用户年龄。同样，您可以决定如何构建观察值和奖励。在预测中，封装在预测请求中的观察值再次是与训练中相同类型的用户向量，具有相同的真实世界含义；您可以用相同的机制生成它们。

您的预测目标再次是确定为特定用户推荐哪些电影项目。您会使用您确定的机制用用户向量代表该用户，将该向量作为观察值发送，并在响应中获取推荐的电影项目。

### 性能和可扩展性分析

你可以使用TensorBoard Profiler，以及其他TensorBoard功能，来分析训练性能并找到加速和/或更好扩展应用程序的解决方案。

## 清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于此教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Delete endpoint resource
! gcloud ai endpoints delete $endpoint.name --quiet --region $REGION

# Delete model resource
! gcloud ai models delete $model.name --quiet

# Delete Cloud Storage objects that were created
! gsutil -m rm -r $ARTIFACTS_DIR